<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/ConversationEntityMemory%E5%AF%B9%E8%AF%9D%E7%B2%BE%E5%87%86%E6%8F%90%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#!pip install --upgrade --quiet  pillow
#!pip install langchain langchain_community
#!pip install --upgrade httpx httpx-sse PyJWT

from langchain_community.chat_models import ChatZhipuAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

model = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [ ]:
#对话引导提示模板
from langchain import PromptTemplate
SUMMARIZER_TEMPLATE="""请将以下内容逐步概括所提供的对话内容，并将新的橛括添加到之前的概括中，形成新的概括。

EXAMPLE
Current summary:
Human询问AI对人工智能的看法。AI认为人工智能是一种积极的力量。
New lines of conversation:
Human:为什么你认为人工智能是一种积极的力量?
AI:因为人工智能将帮助人类发挥他们的潜能。

New summary:
Human询问AI对人]ǐ智能的看法。AI认为人工智能是一种积极的力量，因为它将帮助人类发挥他们的潜能。
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:"""

SUMMARY_PROMPT = PromptTemplate(
  input_variables=["summary","new_lines"],
  template=SUMMARIZER_TEMPLATE
)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=model,prompt=SUMMARY_PROMPT,max_token_limit=40)
#手动存入历史记录
memory.save_context({"input": "你好"}, {"output": "你好，我是客户欢欢，有什么可以帮助您的么"})
memory.save_context({"input": "我是黑金，我在你们这下了一个订单，订单号是B20460613A,我的邮箱是abc123@bcd.com.前天下的订单还没有收到货，请帮助查一下"},
 {"output": "好的。正在为查询您的订单"})

In [ ]:
#打印的小结
memory.load_memory_variables({})

{'history': 'System: Human与AI打招呼，AI自我介绍为客户欢欢，并表示愿意提供帮助。在Human表达了想了解售后服务的意愿后，AI说明了提供7*24小时的在线客服支持。随后，Human作为黑金客户，提供了自己的订单号和邮箱，并请求AI帮助查询前天下的订单B20460613A为何还未收到货的情况。 \n\nNew summary: \nHuman与AI客户欢欢打招呼，并询问售后服务。AI介绍了全天候在线客服支持，并在黑金客户提供订单号B20460613A和邮箱abc123@bcd.com后，被请求帮助查询未收货的订单情况。\nAI: 好的。正在为查询您的订单'}

In [ ]:
#第二部分
#以上只是总结，概述太多。没有突出重点
#下面的代码就是解决记住客户的重点，如邮箱，手机号，订单号
#ConversationEntityMemory

In [15]:
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

entityMemory=ConversationEntityMemory(llm=model)

conversation=ConversationChain(
  llm=model,
  verbose=True,
  prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
  memory=entityMemory
)

conversation.predict(input="""我是黑金，我在你们这下了一个订单，订单号是B20460613A,我的邮箱是abc123@bcd.com.
我的电话是13972647001。前天下的订单还没有收到货，请帮助查一下。如果查到结果白天请打电话，晚上请回复我邮件。""")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'您好，黑金先生/女士，感谢您提供订单信息。我会立即帮您查询订单号B20460613A的状态。请您稍等片刻。\n\n（假设查询过程）\n\n我已经查询到您的订单状态。如果是在正常处理时间内，通常订单会在下单后的1-3天内发出。如果订单有任何异常，我会在确认情况后按照您的要求，在白天给您打电话或者在晚上通过邮件回复您。\n\n如果现在是白天，我将会在接下来的小时内与您取得电话联系。如果是晚上的话，我会在今晚通过邮箱abc123@bcd.com向您报告查询结果。\n\n请您放心，我们会尽快解决您的问题。感谢您的耐心等待和选择我们的服务。\n\n（注：由于我是一个人工智能助手，实际上无法进行真实的订单查询或电话/邮件通讯。上述回复仅模拟实际场景下的客服交流。）'